# Перевод размеченного датасета из русского языка (collection_3) на якутский.
Сам датасет можно скачать по ссылке - http://files.deeppavlov.ai/deeppavlov_data/collection3_v2.tar.gz

Для перевода необходимо получить ключ к API Яндекс-переводчика по этой ссылке - https://translate.yandex.ru/developers/keys

In [247]:
import pandas as pd
import numpy as np
import os
from translater import translate_me
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer
import copy

In [42]:
# Нужен ключ
KEY = 'FIRST_API_KEY'

In [68]:
MAX_SENTENCE_LEN = 700

In [2]:
tokenizer = RussianTokenizer(lowercase=False, alphas_only=False)

In [2]:
delimeter = 'разделитель ||| араарааччы\n'

In [33]:
def delete_unnecessary_lines(filename):
    with open(filename) as f:
        s = f.read()
    s = s.replace('<DOCSTART>\n\n', '')
    s = s.replace('\n\n\n', '\n\n')
    with open(filename, 'w') as f:
        f.write(s)

In [69]:
def translate_file(filename, add_delimiter=False, train_first_part=False, api_key=None):
    with open(filename) as f:
        s = f.read()
    sentences = s.split('\n\n')
    if train_first_part and 'train.txt' in filename:
        sentences = sentences[:7000]
    if not train_first_part and 'train.txt' in filename:
        sentences = sentences[7000:]
    corpus = ''
    if add_delimiter:
        corpus = delimeter
    for sentence in sentences:
        try:
            words = [line.split()[0] for line in sentence.split('\n')]
            text = ' '.join(words)
            entities = [line.split()[1] for line in sentence.split('\n')]
            if len(text) > MAX_SENTENCE_LEN: # Яндекс-переводчик почему-то не переводит длинные тексты
                continue
        except IndexError:
            continue
        tokens = tokenizer([translate_me(text, api_key)])[0]
        corpus += ' '.join(words) + ' ||| ' + ' '.join(tokens) + '\n'
        
    with open('data/corpus.txt', 'a') as f:
        f.write(corpus)

## Перевод текстов
Переводим тексты в формате - {русский текст} ||| {якутский текст}, как указано в https://github.com/clab/fast_align
Между файлами будет разделитель - "**разделитель ||| араарааччы**"

**Обратите внимание** на то, что АПИ сервиса позволяет в день перевести только 1.000.000 символов в день, в файле train около 1.200.000 символов, valid и train по 300.000. Поэтому нужно либо подождать день после перевода первой части train, либо получить дополнительный ключ с другого аккаунта.


In [35]:
delete_unnecessary_lines('data/collection_3/train.txt')
delete_unnecessary_lines('data/collection_3/valid.txt')
delete_unnecessary_lines('data/collection_3/test.txt')

In [21]:
translate_file('data/train.txt', add_delimiter=False, train_first_part=True, api_key=KEY)

In [ ]:
KEY = 'SECOND_KEY'

In [ ]:
translate_file('data/collection_3/train.txt', add_delimiter=False, train_first_part=False, api_key=KEY)
translate_file('data/collection_3/valid.txt', add_delimiter=True, train_first_part=False, api_key=KEY)
translate_file('data/collection_3/test.txt', add_delimiter=True, train_first_part=False, api_key=KEY)

## Обработка переведенного текста
В нашей токенизации пунктуационные знаки считаются за отдельные токены, после перевода к словам могут прикрепиться символы '.', '-'

In [44]:
def clear_text(text, split_sign):
    words = []
    for word in text.split():
        if word != split_sign:
            words.append(word.split(split_sign)[0])
        if word[-1] == split_sign:
            words.append(split_sign)
    return ' '.join(words)

In [45]:
new_corpus = ''
with open('data/corpus.txt') as f:
    for line in f:
        left, right = line.split(' ||| ')
        left = left + ' ||| '
        right = clear_text(right, split_sign='.')
        right = clear_text(right, split_sign='-')
        left = left + right + '\n'
        new_corpus += left
        
with open('data/cleaned_corpus.txt', 'w') as f:
    f.write(new_corpus)

# Сопоставление слов с текстов русского языка на переведенные якутские.

установить https://github.com/clab/fast_align

1. Сопоставить с русского на якутский.
```
./fast_align -i data/cleaned_corpus.txt -d -o -v > data/forward.align
```
2. Сопоставить с якутского на русский.
```
./fast_align -i data/cleaned_corpus.txt -d -o -v -r > data/reverse.align
```
3. Применить эвристику с симметричными сопоставлениями для улучшения качества
```
./atools -i data/forward.align -j data/reverse.align -c data/grow-diag-final-and
```

In [33]:
# Примеры сопоставлений
with open('data/cleaned_corpus.txt') as f:
    corpus = f.read().split('\n')
with open('data/final.align') as f:
    mapping = f.read().split('\n')
    
for i in range(10):
    left, right = corpus[i].split(' ||| ')
    left = left.split()
    right = right.split()
    print(corpus[i])
    for ij in mapping[i].split():
        ii, jj = list(map(int, ij.split('-')))
        print(left[ii], right[jj])
    print('\n')
    

Дополнение : Д . Медведев присвоил звания сотрудников полиции и переназначил 14 руководителей УВД , ГУВД и МВД по субъектам РФ . ||| Эбии : Д . Медведев полиция үлэһиттэрин аатын - суолун ылан РФ ИДьМ , ГУВД , ИДьМ субъектарыгар ИДьМ салайааччыларын ыҥырда .
Дополнение Эбии
: :
Д Д
. .
Медведев Медведев
присвоил полиция
звания аатын
сотрудников үлэһиттэрин
полиции полиция
и суолун
и ылан
переназначил РФ
руководителей салайааччыларын
УВД ИДьМ
, ,
, ,
ГУВД ГУВД
МВД ИДьМ
по ИДьМ
субъектам субъектарыгар
. .


Президент России Дмитрий Медведев присвоил звания сотрудников полиции и переназначил 14 руководителей управлений , главных управлений и министерств внутренних дел по субъектам РФ и федеральным округам . ||| Россия Президенэ Дмитрий Медведев полиция үлэһиттэрин аатын иҥэрдэ уонна РФ субъектарыгар уонна федеральнай уокуруктарга ис дьыала министерствотын 14 управлениеларыгар , кылаабынай управлениеларга уонна ис дьыала министерстволарыгар 14 салайааччыны анаата .
Президент Президенэ
Росс

In [185]:
def extract_translated_text(corpus):
    corpus = corpus.strip()
    sentences = corpus.split('\n')
    sentences = [sentence.split(' ||| ')[1].split() for sentence in sentences]
    return sentences


def extract_annotations(filename):
    with open(filename) as f:
        s = f.read().strip()
    sentences = s.split('\n\n')
    new_sentences = [[line.split()[0] for line in sentence.split('\n')] for sentence in sentences]
    annotations = [[line.split()[1] for line in sentence.split('\n')] for sentence in sentences]
    filter_length = [len(' '.join(sentence)) <= MAX_SENTENCE_LEN for sentence in new_sentences]
    annotations = np.array(annotations)[filter_length].tolist()
    return annotations


def get_sentences_length(annotations):
    len_sentences = [len(sentence) for sentence in annotations]
    return len_sentences

In [186]:
with open('data/cleaned_corpus.txt') as f:
    corpus = f.read().strip()
train = extract_translated_text(corpus.split(delimeter)[0])
valid = extract_translated_text(corpus.split(delimeter)[1])
test = extract_translated_text(corpus.split(delimeter)[2])

train_annotations = extract_annotations('data/collection_3/train.txt')
valid_annotations = extract_annotations('data/collection_3/valid.txt')
test_annotations = extract_annotations('data/collection_3/test.txt')

train_len_sentences = get_sentences_length(train_annotations)
valid_len_sentences = get_sentences_length(valid_annotations)
test_len_sentences = get_sentences_length(test_annotations)

In [187]:
def create_align_mapping(align, len_sentences):
    align_mapping = [[[] for _ in range(len_sentences[i])] for i in range(len(align))]
    for i_sentence, sentence in enumerate(align):
        for mapping in sentence.split():
            left, right = list(map(int, mapping.split('-')))
            align_mapping[i_sentence][left].append(right)
    return align_mapping

In [188]:
# Считываем сопоставления
with open('data/final.align') as f:
    align = f.read().split('\n')
    
train_align = align[:len(train)]
valid_align = align[len(train) + 1:len(train) + len(valid) + 1]
test_align = align[len(train) + len(valid) + 2:]
test_align = test_align[:len(test)] # generated file has empty line at the end

train_align_mapping = create_align_mapping(train_align, train_len_sentences)
valid_align_mapping = create_align_mapping(valid_align, valid_len_sentences)
test_align_mapping = create_align_mapping(test_align, test_len_sentences)

In [263]:
def annotate_translated_sentences(annotations, translated_sentences, align_mapping):
    translated_annotations = [['O' for _ in sentence] for sentence in translated_sentences]
    n_sentences = len(annotations)
    for i_sen in range(n_sentences):        
        for i_left, annotation in enumerate(annotations[i_sen]):
            for i_right in align_mapping[i_sen][i_left]:
                translated_annotations[i_sen][i_right] = annotations[i_sen][i_left]
    return translated_annotations


def correct_annotation(sentences, annotations):
    new_annotations = copy.deepcopy(annotations)
    begin = False
    rem = False
    for i_sen, words in enumerate(sentences):
        for i in range(len(new_annotations[i_sen])):
            if new_annotations[i_sen][i] == 'O':
                begin = False
                rem = False
            if not begin and new_annotations[i_sen][i] != 'O':
                new_annotations[i_sen][i] = 'B-' + new_annotations[i_sen][i].split('-')[1]
            if new_annotations[i_sen][i][0] == 'B':
                begin = True
            if new_annotations[i_sen][i][0] == 'B' and words[i][0].isalpha() and words[i][0].islower():
                rem = True
            if new_annotations[i_sen][i][0] == 'B' and not (words[i][0].isalpha() and words[i][0].islower()):
                rem = False
            if rem:
                new_annotations[i_sen][i] = 'O'
    return new_annotations

In [264]:
train_translated_annotations = annotate_translated_sentences(train_annotations, train, train_align_mapping)
valid_translated_annotations = annotate_translated_sentences(valid_annotations, valid, valid_align_mapping)
test_translated_annotations = annotate_translated_sentences(test_annotations, test, test_align_mapping)

train_translated_annotations = correct_annotation(train, train_translated_annotations)
valid_translated_annotations = correct_annotation(valid, valid_translated_annotations)
test_translated_annotations = correct_annotation(test, test_translated_annotations)

In [275]:
new_train = [['\t'.join(pair) for pair in  zip(train[i], train_translated_annotations[i])] for i in range(len(train))]
new_valid = [['\t'.join(pair) for pair in  zip(valid[i], valid_translated_annotations[i])] for i in range(len(valid))]
new_test = [['\t'.join(pair) for pair in  zip(test[i], test_translated_annotations[i])] for i in range(len(test))]

In [276]:
new_train = '\n\n'.join(['\n'.join(sentence) for sentence in new_train])
new_valid = '\n\n'.join(['\n'.join(sentence) for sentence in new_valid])
new_test = '\n\n'.join(['\n'.join(sentence) for sentence in new_test])

In [279]:
with open('data/collection_3/train_sa.txt', 'w') as f:
    f.write(new_train)
with open('data/collection_3/valid_sa.txt', 'w') as f:
    f.write(new_valid)
with open('data/collection_3/test_sa.txt', 'w') as f:
    f.write(new_test)